**Prerequisites**
- Git repository that contains this project (push your local changes somewhere accessible).
- Hugging Face access token with permission to pull SAM3 weights.
- Kaggle API token (kaggle.json) if you want the notebook to download the fruits dataset automatically, otherwise mount Google Drive and point to an existing copy of `data/fruits`.
- Colab runtime set to GPU (Runtime → Change runtime type → Hardware accelerator → GPU).

In [5]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
#@title Runtime configuration
REPO_URL = "https://github.com/rishii2208/sam3_pipeline.git"  #@param {type:"string"}
PROJECT_DIR = "/content/readme"  #@param {type:"string"}
MOUNT_DRIVE = False  #@param {type:"boolean"}
DRIVE_DATASET_PATH = "/content/drive/MyDrive/datasets/fruits"  #@param {type:"string"}
DOWNLOAD_WITH_KAGGLE = True  #@param {type:"boolean"}
TRAIN_DEVICE = "cuda"  #@param ["cuda", "cpu"]
CALIBRATION_LIMIT = 0  #@param {type:"integer"}
EVAL_LIMIT = 0  #@param {type:"integer"}
PIPELINES = ["p2", "p3"]
assert REPO_URL, "Set REPO_URL to your Git repository URL."

In [23]:
%cd /content/readme
!python scripts/train.py --pipeline p2 --device cuda --limit 10

/content/readme
Traceback (most recent call last):
  File "/content/readme/scripts/train.py", line 5, in <module>
    from src.pipelines.runner import PipelineRunner
ModuleNotFoundError: No module named 'src'


In [24]:
%cd /content/readme
import os
os.environ["PYTHONPATH"] = "/content/readme"
!python scripts/train.py --pipeline p2 --device cuda --limit 10

/content/readme
Traceback (most recent call last):
  File "/content/readme/scripts/train.py", line 5, in <module>
    from src.pipelines.runner import PipelineRunner
  File "/content/readme/src/pipelines/runner.py", line 13, in <module>
    from src.data.dataset import FruitsDataset
ModuleNotFoundError: No module named 'src.data'


In [26]:
%cd /content/readme
!PYTHONPATH=/content/readme python scripts/train.py --pipeline p2 --device cuda --limit 10

/content/readme
Traceback (most recent call last):
  File "/content/readme/scripts/train.py", line 5, in <module>
    from src.pipelines.runner import PipelineRunner
  File "/content/readme/src/pipelines/runner.py", line 13, in <module>
    from src.data.dataset import FruitsDataset
ModuleNotFoundError: No module named 'src.data'


In [7]:
#@title Enter authentication tokens (input hidden after execution)
import getpass
HF_TOKEN = getpass.getpass("Hugging Face token (leave blank to skip): " ).strip()
print("Token captured?", bool(HF_TOKEN))

Token captured? True


NameError: name 'MOUNT_DRIVE' is not defined

In [8]:
#@title Clone (or re-clone) the repository
import pathlib
import shutil
project_path = pathlib.Path(PROJECT_DIR)
if project_path.exists():
    print(f'Removing existing directory: {project_path}')
    shutil.rmtree(project_path)
!git clone {REPO_URL} {PROJECT_DIR}
%cd {PROJECT_DIR}

Cloning into '/content/readme'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 1), reused 29 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 15.03 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/readme
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 29 (delta 1), reused 29 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (29/29), 15.03 KiB | 3.00 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/readme


In [9]:
#@title Install Python dependencies (includes SAM3 + Triton)
%pip install -q -r requirements.txt
%pip install -q -e external/sam3[train]
%pip install -q triton==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 874.1 kB/s eta 0:00:0000:0100:01   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/779.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 874.1 kB/s eta 0:00:0000:0101
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 115.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 115.1 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 22.8 MB/

In [10]:
#@title Authenticate with Hugging Face
if HF_TOKEN:
    from huggingface_hub import login
    login(token=HF_TOKEN, add_to_git_credential=False)
else:
    print('HF token not provided; assuming checkpoints already cached.')

In [16]:
from pathlib import Path
cred_path = Path.home() / "kaggle.json"
print("Credential file exists?", cred_path.exists())
if cred_path.exists():
    cred_path.chmod(0o600)

Credential file exists? False


In [17]:
#@title Prepare the fruits dataset
import json
import os
import zipfile

from pathlib import Path
import shutil

project_path = Path(PROJECT_DIR)
data_root = project_path / 'data' / 'fruits'
data_root.mkdir(parents=True, exist_ok=True)
home_kaggle_dir = Path.home() / '.kaggle'
home_kaggle_dir.mkdir(exist_ok=True)
home_cred_path = home_kaggle_dir / 'kaggle.json'
repo_cred_path = project_path / '.kaggle' / 'kaggle.json'
if repo_cred_path.exists():
    shutil.copy2(repo_cred_path, home_cred_path)
if not home_cred_path.exists():
    default_creds = {
        "username": "anup222222",
        "key": "54c5b8a4a6ddd30a1523fe2fb7b60665",
    }
    home_cred_path.write_text(json.dumps(default_creds), encoding='utf-8')
home_cred_path.chmod(0o600)
if DOWNLOAD_WITH_KAGGLE:
    %pip install -q kaggle
    !kaggle datasets download afsananadia/fruits-images-dataset-object-detection -p {data_root} -o
    zip_path = data_root / 'fruits-images-dataset-object-detection.zip'
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, 'r') as archive:
            archive.extractall(data_root)
else:
    source_path = Path(DRIVE_DATASET_PATH)
    if source_path.exists():
        print(f'Copying dataset from {source_path} ...')
        if data_root.resolve() != source_path.resolve():
            shutil.copytree(source_path, data_root, dirs_exist_ok=True)
    else:
        print('Drive dataset path missing; ensure data/fruits is populated manually.')

Dataset URL: https://www.kaggle.com/datasets/afsananadia/fruits-images-dataset-object-detection
License(s): CC0-1.0
  7% 5.00M/70.2M [00:00<00:02, 32.2MB/s]Downloading fruits-images-dataset-object-detection.zip to /content/readme/data/fruits
 85% 60.0M/70.2M [00:00<00:00, 88.9MB/s]
100% 70.2M/70.2M [00:00<00:00, 93.4MB/s]
 85% 60.0M/70.2M [00:00<00:00, 88.9MB/s]
100% 70.2M/70.2M [00:00<00:00, 93.4MB/s]


In [18]:
#@title Build train/val/test splits (idempotent)
import pathlib
split_index = pathlib.Path(PROJECT_DIR) / 'data' / 'fruits' / 'splits.json'
if split_index.exists():
    print(f'Splits already exist at {split_index}')
else:
    !python scripts/split_data.py data/fruits --train-ratio 0.7 --val-ratio 0.15

Wrote split metadata to data/fruits/splits.json


In [29]:
%cd /content/readme


/content/readme


In [31]:
%cd /content/readme
!python scripts/train.py --pipeline p2 --device cuda --limit 10

/content/readme
Traceback (most recent call last):
  File "/content/readme/scripts/train.py", line 5, in <module>
    from src.pipelines.runner import PipelineRunner
  File "/content/readme/src/pipelines/runner.py", line 13, in <module>
    from src.data.dataset import FruitsDataset
ModuleNotFoundError: No module named 'src.data'


In [27]:
#@title Train + evaluate pipelines
import os
import shlex
import subprocess
cal_limit = CALIBRATION_LIMIT if CALIBRATION_LIMIT > 0 else None
eval_limit = EVAL_LIMIT if EVAL_LIMIT > 0 else None
env = os.environ.copy()
env['PYTHONPATH'] = PROJECT_DIR
def run_cmd(cmd: str):
    print(f'\n>>> {cmd}')
    subprocess.run(shlex.split(cmd), cwd=PROJECT_DIR, env=env, check=True)
for pipeline in PIPELINES:
    train_cmd = f'python scripts/train.py --pipeline {pipeline} --device {TRAIN_DEVICE}'
    if cal_limit:
        train_cmd += f' --limit {cal_limit}'
    run_cmd(train_cmd)
    eval_cmd = f'python scripts/evaluate.py --pipeline {pipeline} --device {TRAIN_DEVICE} --split val'
    if eval_limit:
        eval_cmd += f' --limit {eval_limit}'
    run_cmd(eval_cmd)


>>> python scripts/train.py --pipeline p2 --device cuda


CalledProcessError: Command '['python', 'scripts/train.py', '--pipeline', 'p2', '--device', 'cuda']' returned non-zero exit status 1.

In [ ]:
#@title Inspect generated result artifacts
from pathlib import Path
results_dir = Path(PROJECT_DIR) / 'results'
if not results_dir.exists():
    print('No results directory yet.')
else:
    for artifact in sorted(results_dir.rglob('*.json')):
        rel_path = artifact.relative_to(Path(PROJECT_DIR))
        print(rel_path)